In [1]:
import os
import sys
import time
from datetime import datetime
from datetime import timedelta
from itertools import chain
from itertools import cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg

#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [2]:
pwd

'/dados/home/rsouza/exploreAT-Topothek.NLP'

Reading the XML Metadata files:

In [3]:
readme =  './TEI-XML-2018/README_English.txt'
print((open(readme, encoding="latin-1").read()))


Content of the XML files
XML are partially enriched with location data
* The location data are available as a tree in the respective entries
* Location data are available for those entries specified in helper_tables\placeName-bis Freq 10-nur Ö+STir - Tabellenblatt1.csv
* The basis are complete matches in the tag placeName

The complete place hierarchy can be found in helper_tables\listPlace.xml. This resulted in helper_tables\listPlaceBySigle.xml

There could be manual corrections to the data.


Processing Information on Places

In [4]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [5]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [6]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [7]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [9]:
identified_places = collections.defaultdict(list)
def show_record(something):
    if isinstance(something, list):
        for elem in something:
            show_record(elem)
    elif isinstance(something, collections.OrderedDict) and ('record' in something.keys()):
        try:
            #print(something['record']['Bundesland'], ' --> ', something['record']['Ort'])
            identified_places[something['record']['Bundesland']].append(something['record']['Ort'])
        except:
            #print(something['record'][0]['Bundesland'], ' --> ', something['record'][0]['Ort'])
            identified_places[something['record'][0]['Bundesland']].append(something['record'][0]['Ort'])
    elif isinstance(something, collections.OrderedDict):
        new = something['listPlace']['place']
        show_record(new)
    else:
        print(type(something))
        print(something)

In [10]:
show_record(placeslist)

In [11]:
identified_places

defaultdict(list,
            {'Bgl.': ['Deutsch Jahrndorf',
              'Edelstal',
              'Gattendorf',
              'Kittsee',
              'Neudorf, Novo Selo',
              'Nickelsdorf',
              'Pama, Bijelo Selo',
              'Parndorf, Pandrof',
              'Potzneusiedl',
              'Zurndorf',
              'Gols',
              'Halbturn',
              'Mönchhof',
              'Neusiedl am See',
              'Podersdorf am See',
              'Weiden am See',
              'Andau',
              'Apetlon',
              'Frauenkirchen',
              'Illmitz',
              'Pamhagen',
              'St. Andrä am Zicksee',
              'Tadten',
              'Wallern im Burgenland',
              'Antau, Otava',
              'Bad Sauerbrunn',
              'Breitenbrunn',
              'Bruckneudorf',
              'Donnerskirchen',
              'Großhöflein',
              'Hirm',
              'Hornstein, Vorištan',
              'Jois',
 

In [11]:
print(identified_places.keys())

dict_keys(['STir.', 'Kä.', 'OÖ', 'OTir.', 'W.', 'NTir.', 'NÖ', 'Sa.', 'St.', 'Bgl.'])


In [15]:
geolocator = gg.Nominatim(user_agent="test_application")

In [16]:
logradouro = identified_places['Bgl.'][0]
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

Deutsch Jahrndorf, Bezirk Neusiedl, Burgenland, 2423, Österreich
(48.0116888, 17.1069611)
{'place_id': '197784614', 'osm_id': '303140', 'display_name': 'Deutsch Jahrndorf, Bezirk Neusiedl, Burgenland, 2423, Österreich', 'lat': '48.0116888', 'lon': '17.1069611', 'type': 'administrative', 'boundingbox': ['47.9708775', '48.031224', '17.053944', '17.160776'], 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png', 'osm_type': 'relation', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'importance': 0.523562468688337, 'class': 'boundary'}


In [17]:
#https://developers.google.com/maps/documentation/staticmaps/
#http://stackoverflow.com/questions/2660201/what-parameters-should-i-use-in-a-google-maps-url-to-go-to-a-lat-lon/9919251#9919251
#m – normal map k – satellite h – hybrid p – terrain

def gmap(address,lat,lon,zoom=15,tmap='m'):
    # Google Maps URL template for an iframe
    google_maps_url = 'http://maps.google.com/maps?q={0}&loc:{1}+{2}&z={3}&t={4}&output=embed'.format(address,
                                                                                                     lat,
                                                                                                     lon,
                                                                                                     zoom,
                                                                                                     tmap,)
    display(IFrame(google_maps_url, '800px', '600px'))
    
gmap(address, latitude, longitude,17,'k')

Reading the XML Data files:

In [12]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [13]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [14]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x7fe12102a458>


In [15]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [16]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [17]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [18]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = Counter(places)
freq_places.most_common()

[('Bundesland', 891),
 ('Großregion', 797),
 ('Kleinregion', 783),
 ('Gemeinde', 696),
 ('Ort', 696)]

Dictionary Approach

In [19]:
xml2dictfile = xmltodict.parse((open(xmlfile, mode='r', encoding='utf-8').read()))

In [20]:
print(xml2dictfile['TEI'].keys())

odict_keys(['@xmlns', 'teiHeader', 'text'])


In [21]:
xml2dictfile['TEI']['@xmlns']
#xml2dictfile['TEI']['teiHeader']

'http://www.tei-c.org/ns/1.0'

In [22]:
print(len(xml2dictfile['TEI']['text']['body']['entry']))
xml2dictfile['TEI']['text']['body']['entry'][0].keys()

1991


odict_keys(['@xml:id', '@xml:lang', 'form', 'gramGrp', 'sense', 'cit', 'note', 'ref', 'etym', 'usg'])

In [23]:
xml2dictfile['TEI']['text']['body']['entry'][0]

OrderedDict([('@xml:id', 'd157_qdb-d1e2'),
             ('@xml:lang', 'bar'),
             ('form',
              [OrderedDict([('@type', 'hauptlemma'), ('orth', 'Tin')]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '1'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                                           ('#text', '|A t~-in')]),
                              OrderedDict([('@notation', 'ipa'),
                                           ('@resp', '#JB'),
                                           ('@change', '01'),
                                           ('#text', '|A t~-in')])]),
                            ('gramGrp', OrderedDict([('gram', '[m,sg+U]')]))]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '2'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                              

In [31]:
for idx, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
    #print(idx)
    places = []
    lemmas = []
    forms = []
    if 'usg' in entry.keys():
        if isinstance(entry['usg'], list):
            for local in entry['usg']:
                try:
                    #print(local['placeName']['#text'])
                    places.append(local['placeName']['#text'])
                except:
                    #print(local['placeName'])
                    places.append(local['placeName'])
        else:
            places = []
            try:
                #print(entry['usg']['placeName']['#text']);
                places.append(entry['usg']['placeName']['#text'])
            except:
                #print(entry['usg']['placeName']);
                places.append(entry['usg']['placeName'])
                
    if 'form' in entry.keys():            
        if isinstance(entry['form'], list):
            for elem in entry['form']:
                if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                    #print(elem['orth'])
                    lemmas.append(elem['orth'])
                elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                    for subelem in elem['orth']:
                        if subelem['@type'] == 'normalized':
                            #print(subelem['#text'])
                            lemmas.append(subelem['#text'])
        else:
            if isinstance(entry['form']['orth'], list):
                for elem in entry['form']['orth']:
                    if elem['@type'] == 'normalized':
                        #print(elem['#text'])
                        lemmas.append(elem['#text'])
            else:
                #print(entry['form']['orth'])
                lemmas.append(entry['form']['orth'])
    #if 'cit' in entry.keys():
    #    print(entry['cit']) #['ref'])
    #    print(entry['cit'].keys())
  
    print(lemmas, places)

['Tin'] ['Matrei/O. OTir.']
['Häutin'] ['Matrei/O. OTir.']
['Gesotttin'] ['Matrei/O. OTir.']
['Stadel'] ['Matrei/O. OTir.']
['Tin'] ['Matrei/O. OTir.']
['Stadeltor'] ['Matrei/O. OTir.']
['Tina'] ['Salurn']
['Tina'] ['Salurn']
['Dinamít'] ['(Umg.) Herzogenb-g. NÖ']
['Dinamít'] []
['Dinamít'] []
['Dinamít'] []
['Dinamo'] []
['Dinamo'] ['(Umg.) Herzogenb-g. NÖ']
['Dinamo'] ['Königswerth Egerld.']
['Tina'] ['Marling']
['Tina'] ['Marling']
['Tina'] ['Marling']
['Dinastí'] ['(Umg.) Herzogenb-g. NÖ']
['dinástisch'] ['(Umg.) Herzogenb-g. NÖ']
['Tinazia'] []
['tindern'] []
['tindern'] []
['tindern'] []
['tindern'] ['Malta Kä.']
['tindern'] ['Malta Kä.']
['schebern'] ['Malta Kä.']
['scheppern'] ['Malta Kä.']
['tuschen'] ['Malta Kä.']
['Tinef'] ['Wien']
['Tinef'] ['Wien']
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] []
['Tinef'] ['Wien']
['Schmarren'] ['Wien']
['Drëck'] ['Wien']
['Tinef'] []
['Tinéll'] []
['Tinellein'] ['Böwald'

['guotding'] ['Natternb. OÖ']
['guotding'] []
['guotding'] ['Prambachkn. OÖ']
['guotding'] ['Kl.Piesenham in Gm. Pramet OÖ (Herk.)']
['guotding'] []
['guotding'] []
['guotding'] []
['guotding'] ['Götzendf. in Gm. Oepping OÖ']
['guotding'] ['Schlagen in Gm. Gmunden OÖ']
['guotding'] []
['guotding'] []
['guotding'] ['(Umg./Bz.) Braunau OÖ']
['guotding'] ['(Umg./Bz.) Braunau OÖ']
['guotding'] ['Mittersill Sa.']
['guotding'] ['Kröllendf. NÖ']
['guotding'] ['St.Veit/M. OÖ']
['guotding'] []
['guotding'] []
['guotding'] ['Gmunden OÖ']
['guotding'] ['Thening in Gm. Kirchbg.-Thening OÖ']
['guotding'] ['Thening in Gm. Kirchbg.-Thening OÖ']
['guotding'] ['Thening in Gm. Kirchbg.-Thening OÖ']
['guotding'] ['Dürrnbg. in Gm. Hallein Sa.']
['guotding'] ['Nauders NTir.?']
['guotding'] ['(Umg./Bz.) Braunau OÖ']
['guotding'] []
['guotding'] []
['guotding'] ['Thiers. NTir. undNiederndf./NTir.']
['guotding'] ['Mühlhm. OÖ']
['guotding'] ['(Umg./Bz.) Braunau OÖ']
['guotding'] ['Mühlhm. OÖ']
['guotding'] ['(

['ausdingen'] []
['ausdingen'] ['Ma.Luggau Kä.']
['ausdingen'] ['Wr. Neustadt']
['ausdingen'] ['Gastern NÖ']
['ausdingen'] ['Braunau OÖ']
['ausdingen'] ['Kitzbühel NTir.']
['bedingen'] []
['bedingen'] []
['bedingen'] []
['bedingen'] []
['forbedingen'] []
['anbedingen'] []
['ausbedingen'] []
['ausbedingen'] ['Ulrichsbg. OÖ']
['unbedingen'] []
['eindingen'] ['Waltrowitz SMä. (Herk.)']
['eindingen'] ['Welschn.']
['eindingen'] ['Welschn.']
['eindingen'] []
['eindingen'] []
['eindingen'] []
['eindingen'] []
['eindingen'] []
['eindingen'] ['Hochfilzen']
['eindingen'] []
['eindingen'] ['Waltrowitz SMä. (Herk.)']
['ferdingen'] []
['ferdingen'] ['VUMB in NÖ ^#^# tw. LT-O']
['dingen'] ['VUMB in NÖ ^#^# tw. LT-O', 'Röschitz']
['ferdingen'] ['Ulrichsbg. OÖ']
['ferdingen'] ['Pongau']
['ferdingen'] []
['ferdingen'] ['Gr.Tajax SMä.']
['ferdingen'] []
['ferdingen'] ['Michaelbeuern * 17. jh.']
['ferdingen'] []
['ferdingen'] ['Dobrzan Egerld.']
['ferdingen'] []
['ferdingen'] []
['ferdingen'] []
['ferdin

['Gallapfeltinte'] ['Kufstn. NTir.']
['Gallapfeltinte'] ['Radschin in Gm. Kautzen NÖ [2o/1.]']
['Gallapfeltinte'] ['Rangersdf. Kä.']
['Tinte'] ['(Leimsgrub u. Moarhäuser in Gm.) Schönbg. BöW']
['Schuolertinte'] ['Julb. OÖ']
['Wachstinte'] []
['tindel\xa0tändel'] ['STir.', 'allg.']
['Tinter'] ['Wien']
['Tinter'] []
['Tinter'] []
['Tinter'] ['Wien']
['tintig'] []
['tintig'] []
['tintig'] ['Weer NTir.']
['tintig'] ['Thiers. NTir.']
['Tintinger'] []
['Tintinger'] []
['Tintinger'] ['?? ^#^# LT-O']
['Tintinger'] ['Gersthf. (im 18.Bz.) in Gm. Wien ^#^# 1x Treibach: ON?']
['Tintinger'] []
['Tintinger'] ['?? Bgl. ^#^# LT-O', 'Neustift b. Schlaining']
['Tintinger'] ['Gersthf. (im 18.Bz.) in Gm. Wien']
['Tintinger'] ['Reidling in Sitzenbg.-Reidling NÖ']
['Tintinger'] ['Prambachkn. OÖ']
['Tintinger'] []
['Tintinger'] ['STir.']
['Tintinger'] []
['Tintinger'] ['Wien']
['Tinte'] ['Wien']
['einhinhupfen'] ['Wien']
['Hopfinger'] ['Wien']
['Tintinger'] ['Fusch Sa.']
['Tintinger'] ['Klagenfurt Kä.']
['Ti

['Mëltirn'] ['Au/Hofkirchen']
['Mëltirn'] []
['Tirn'] []
['austiren'] []
['Tirn'] []
['Tirn'] []
['Tirn'] ['?? Bgl. ^#^# LT-O', 'Stotzing nBgl.']
['Tirn'] []
['Tirnleinpere'] []
['Tirn'] []
['Tirn'] []
['Schnaps'] ['Freild. in Gm. Türnitz NÖ']
['Tirn'] ['Freild. in Gm. Türnitz NÖ']
['Tirn'] ['Türnitz NÖ']
['Tirn'] []
['Tirn'] ['Linz OÖ']
['Tirn'] []
['Tirnleinpere'] []
['Tirnleinkërsche'] []
['Tirnleinpaum'] []
['Tirn'] ['Langenwang']
['Tirn'] []
['tirnlen'] ['Gerolding in Gm. Dunkelsteinerwd. NÖ']
['Tirn'] []
['Tirnleinpere'] []
['Tirnleinholz'] []
['Tirn'] []
['Tirnetze'] ['Ungar. Bergland']
['tirnlen'] ['(Umg.) Türnitz NÖ', 'Mitt.']
['tirnlen'] ['Mittereck in Gm. Türnitz NÖ']
['tirnlen'] ['Puchbg. NÖ']
['tirnlen'] ['Mauer (im 23.Bz.) in Gm. Wien']
['tirnlen'] ['Puchbg. NÖ']
['tirnlen'] []
['Tirnling'] ['Puchbg. NÖ']
['Tirnling'] []
['Tirnling'] ['Miesenb. NÖ']
['Tirnling'] ['Puchbg./S. NÖ']
['Tirnling'] []
['Tirnling'] ['Krumb. NÖ']
['Tirnling'] ['Südmähren']
['Tirnzlein'] []
['Tirn

In [ ]:
places = []
lemmas = []
forms = []
for idx, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
    #print(idx)
    if 'usg' in entry.keys():
        if isinstance(entry['usg'], list):
            for local in entry['usg']:
                try:
                    #print(local['placeName']['#text'])
                    places.append(local['placeName']['#text'])
                except:
                    #print(local['placeName'])
                    places.append(local['placeName'])
        else:
            places = []
            try:
                #print(entry['usg']['placeName']['#text']);
                places.append(entry['usg']['placeName']['#text'])
            except:
                #print(entry['usg']['placeName']);
                places.append(entry['usg']['placeName'])
                
    if 'form' in entry.keys():            
        if isinstance(entry['form'], list):
            for elem in entry['form']:
                if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                    #print(elem['orth'])
                    lemmas.append(elem['orth'])
                elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                    for subelem in elem['orth']:
                        if subelem['@type'] == 'normalized':
                            #print(subelem['#text'])
                            lemmas.append(subelem['#text'])
        else:
            if isinstance(entry['form']['orth'], list):
                for elem in entry['form']['orth']:
                    if elem['@type'] == 'normalized':
                        #print(elem['#text'])
                        lemmas.append(elem['#text'])
            else:
                #print(entry['form']['orth'])
                lemmas.append(entry['form']['orth'])
    #if 'cit' in entry.keys():
    #    print(entry['cit']) #['ref'])
    #    print(entry['cit'].keys())
  
    print(lemmas, places)